In [15]:
import pandas as pd

df = pd.read_csv("/Users/benbiggs/Downloads/archive/fifa_ranking-2024-04-04.csv")
df = df[df['rank_date'] == '2024-04-04'].sort_values('rank').reset_index(drop=True)
df['country_full'] = df['country_full']
print (df)

      rank            country_full country_abrv  total_points  \
0      1.0               Argentina          ARG       1858.00   
1      2.0                  France          FRA       1840.59   
2      3.0                 Belgium          BEL       1795.23   
3      4.0                 England          ENG       1794.90   
4      5.0                  Brazil          BRA       1788.65   
..     ...                     ...          ...           ...   
206  207.0  British Virgin Islands          VGB        813.31   
207  208.0       US Virgin Islands          VIR        797.29   
208  209.0                Anguilla          AIA        793.48   
209  210.0              San Marino          SMR        742.05   
210    NaN                 Eritrea          ERI        855.56   

     previous_points  rank_change confederation   rank_date  
0            1855.20            0      CONMEBOL  2024-04-04  
1            1845.44            0          UEFA  2024-04-04  
2            1798.46           -1

In [16]:
with open('euro24_groups.lns', 'r') as f:
    data = f.readlines()

groups = []
teams_in_group = []
while len(data) > 0:
    entry = data.pop().strip()
    teams_in_group = []
    while 'Group' not in entry:
        teams_in_group.append(entry)
        entry = data.pop().strip()
    
    for t in teams_in_group:
        groups.append({
            'group_name': entry,
            'country_full': t
        })

group_df = pd.DataFrame(groups).sort_values('group_name').reset_index(drop=True)
group_df = group_df.replace('Türkiye', 'Turkey')
# print (group_df)

df_teams = group_df.merge(df, on='country_full', how='left')

# create a euro ranking
df_teams = df_teams.sort_values('rank').reset_index()
df_teams['euro24_ranking'] = df_teams.index + 1

df_teams = df_teams[['group_name', 'country_full', 'country_abrv', 'euro24_ranking', 'rank']]
print (df_teams)


   group_name country_full country_abrv  euro24_ranking  rank
0     Group D       France          FRA               1   2.0
1     Group E      Belgium          BEL               2   3.0
2     Group C      England          ENG               3   4.0
3     Group F     Portugal          POR               4   6.0
4     Group D  Netherlands          NED               5   7.0
5     Group B        Spain          ESP               6   8.0
6     Group B        Italy          ITA               7   9.0
7     Group B      Croatia          CRO               8  10.0
8     Group A      Germany          GER               9  16.0
9     Group A  Switzerland          SUI              10  19.0
10    Group C      Denmark          DEN              11  21.0
11    Group E      Ukraine          UKR              12  22.0
12    Group D      Austria          AUT              13  25.0
13    Group A      Hungary          HUN              14  26.0
14    Group D       Poland          POL              15  28.0
15    Gr

In [17]:
# let's load the players
with open('players.lns', 'r') as g:
    df_players = pd.DataFrame([{
        'name': p.strip()
        }for p in g.readlines()]).reset_index(drop=True)

print (df_players)

         name
0      Adam B
1     Emily P
2     Arjun S
3       Bec L
4       Ben B
5   Grandma B
6   Grandpa B
7    Jackie B
8      Mike L
9      Nick B
10      Pip L
11      Sam B


In [18]:
import numpy as np

np.random.seed(14062024) # start date of euro 2024

tophalf_ids = np.random.permutation(len(df_players))
bottomhalf_ids = np.random.permutation(len(df_players))

df_players_shuffled = pd.concat([df_players.iloc[tophalf_ids], df_players.iloc[bottomhalf_ids]]).reset_index()
print (df_players_shuffled)

    index       name
0       0     Adam B
1       7   Jackie B
2       9     Nick B
3      11      Sam B
4       8     Mike L
5       2    Arjun S
6       5  Grandma B
7       6  Grandpa B
8      10      Pip L
9       3      Bec L
10      1    Emily P
11      4      Ben B
12     10      Pip L
13      0     Adam B
14      7   Jackie B
15      1    Emily P
16      5  Grandma B
17      3      Bec L
18      2    Arjun S
19      9     Nick B
20      6  Grandpa B
21      8     Mike L
22      4      Ben B
23     11      Sam B


In [19]:
# join with the euro teams

df_teams['player'] = df_players_shuffled['name']
print (df_teams.sort_values('group_name'))

   group_name country_full country_abrv  euro24_ranking  rank     player
8     Group A      Germany          GER               9  16.0      Pip L
13    Group A      Hungary          HUN              14  26.0     Adam B
9     Group A  Switzerland          SUI              10  19.0      Bec L
17    Group A     Scotland          SCO              18  39.0      Bec L
5     Group B        Spain          ESP               6   8.0    Arjun S
6     Group B        Italy          ITA               7   9.0  Grandma B
7     Group B      Croatia          CRO               8  10.0  Grandpa B
22    Group B      Albania          ALB              23  66.0      Ben B
2     Group C      England          ENG               3   4.0     Nick B
10    Group C      Denmark          DEN              11  21.0    Emily P
21    Group C     Slovenia          SVN              22  57.0     Mike L
15    Group C       Serbia          SRB              16  33.0    Emily P
4     Group D  Netherlands          NED            

In [20]:
df_teams['printable'] = df_teams.apply(lambda x: f"{x['country_full']} ({x['group_name'].replace('Group ', '')}), rank={x['euro24_ranking']}", axis=1)
teams_per_player = df_teams[['player', 'printable']].groupby('player').agg(list)
teams_per_player = teams_per_player.reset_index()
# print (df_teams[['player', 'printable']].sort_values('player'))
teams_per_player_expanded = pd.DataFrame(
    teams_per_player['printable'].to_list(), 
    index=teams_per_player['player']
).reset_index()

# Rename columns
teams_per_player_expanded.columns = ['player'] + [f'Team_{i+1}' for i in range(2)]

print (teams_per_player_expanded)

       player                    Team_1                 Team_2
0      Adam B        France (D), rank=1   Hungary (A), rank=14
1     Arjun S         Spain (B), rank=6    Turkey (F), rank=19
2       Bec L  Switzerland (A), rank=10  Scotland (A), rank=18
3       Ben B      Ukraine (E), rank=12   Albania (B), rank=23
4     Emily P      Denmark (C), rank=11    Serbia (C), rank=16
5   Grandma B         Italy (B), rank=7   Czechia (F), rank=17
6   Grandpa B       Croatia (B), rank=8  Slovakia (E), rank=21
7    Jackie B       Belgium (E), rank=2    Poland (D), rank=15
8      Mike L   Netherlands (D), rank=5  Slovenia (C), rank=22
9      Nick B       England (C), rank=3   Romania (E), rank=20
10      Pip L       Germany (A), rank=9   Austria (D), rank=13
11      Sam B      Portugal (F), rank=4   Georgia (F), rank=24


In [21]:
print (teams_per_player_expanded.to_markdown())

|    | player    | Team_1                   | Team_2                |
|---:|:----------|:-------------------------|:----------------------|
|  0 | Adam B    | France (D), rank=1       | Hungary (A), rank=14  |
|  1 | Arjun S   | Spain (B), rank=6        | Turkey (F), rank=19   |
|  2 | Bec L     | Switzerland (A), rank=10 | Scotland (A), rank=18 |
|  3 | Ben B     | Ukraine (E), rank=12     | Albania (B), rank=23  |
|  4 | Emily P   | Denmark (C), rank=11     | Serbia (C), rank=16   |
|  5 | Grandma B | Italy (B), rank=7        | Czechia (F), rank=17  |
|  6 | Grandpa B | Croatia (B), rank=8      | Slovakia (E), rank=21 |
|  7 | Jackie B  | Belgium (E), rank=2      | Poland (D), rank=15   |
|  8 | Mike L    | Netherlands (D), rank=5  | Slovenia (C), rank=22 |
|  9 | Nick B    | England (C), rank=3      | Romania (E), rank=20  |
| 10 | Pip L     | Germany (A), rank=9      | Austria (D), rank=13  |
| 11 | Sam B     | Portugal (F), rank=4     | Georgia (F), rank=24  |
